In [ ]:
#change score for pack before last level pack
##calculate relative pack change score
import pandas as pd
import math
address = "results/flink/pack_change_num.csv"
sqrtresult_df = pd.read_csv(address)
for col in sqrtresult_df.columns[1:]:
    sqrtresult_df[col].values[:] = 0

address = "D:/Master/semester3/experiments/features/flink/datedPullFile.csv"
file_df = pd.read_csv(address)
address = "D:/Master/semester3/experiments/features/flink/datedCommitFileAuthor.csv"
commit_df = pd.read_csv(address)
def function_r(x):
    A = x.split("/")
    del A[-1]
    if len(A) > 0:
        del A[-1]
    return('/'.join(A))
file_df['file_path'] = file_df['file_path'].apply(function_r)

for i in range(sqrtresult_df.shape[0]):
    number = sqrtresult_df.loc[i, 'number']
    print(i)
    print(number)
    myfile_df = file_df[file_df.pull_number == number]
    myfile_df = myfile_df.drop_duplicates(subset=['file_path'], keep="first")
    if myfile_df.shape[0] == 0:
        continue
    date = myfile_df.iloc[0, 2]
    for j in range(myfile_df.shape[0]):
        mycommit_df = commit_df[commit_df.date < date]
        path = myfile_df.iloc[j, 1]
        newcommit_df = mycommit_df[mycommit_df.file_path.str.startswith(path)]
        newcommit_df = newcommit_df.drop_duplicates(subset=['oid', 'author_login'], keep="last")
        whole = newcommit_df.shape[0]
        num_df = newcommit_df.groupby(['author_login'])['oid'].count()
        time_df = newcommit_df.groupby(['author_login'])['date'].max()
        for k in range(num_df.shape[0]):
            committer = num_df.index[k]
            num = num_df.iloc[k]
            max_value = time_df.iloc[k]
            if whole != 0:
                lasttime = time_interval(max_value, date)
                if lasttime !=0:
                    sqrtresult_df.loc[i, committer] += (num/whole) / (lasttime**(1/3))
                else:
                    sqrtresult_df.loc[i, committer] += (num/whole) 
sqrtresult_df.to_csv("results/flink/pack2_change_cube.csv", index = False)